In [ ]:
# try again  the external memory approach w/ sklearn style XGboost...!

In [7]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1

In [8]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)


In [7]:
artifactdir = '/opt/program/artifacts/2020-06-15T003722Z' 
bundle = joblib.load(f'{artifactdir}/bundle_2.joblib')
model = bundle['model']
dtest = xgb.DMatrix(f'{artifactdir}/dtest.txt')

y_preds = model.predict(dtest)

[03:35:29] 168683x78 matrix with 168683 entries loaded from /opt/program/artifacts/2020-06-15T003722Z/dtest.txt


In [4]:
bundle = joblib.load('/opt/program/artifacts/2020-06-11T041641Z/bundle.joblib')
model = bundle['model']
param = model.get_xgb_params()
print(param)
print(model.get_num_boosting_rounds()) 
# Oh hmm I was wondering in the "2020-06-12.ipynb" notebook why the results were so bad.
# Perhaps since this model appears to have `100` boosting rounds (in the past I suppse..) maybe indeed
# thiss is a matter of 10 vs 100 rounds.

{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 1, 'missing': nan, 'n_estimators': 100, 'nthread': 1, 'objective': 'multi:softprob', 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': 0, 'subsample': 1, 'verbosity': 1}
100


In [ ]:
dtrain = xgb.DMatrix(f'{artifactdir}/dtrain.txt')
booster = xgb.train({**param, 'num_class': 54}, dtrain, num_boost_round=10)

[03:46:07] 674733x78 matrix with 1349466 entries loaded from /opt/program/artifacts/2020-06-15T003722Z/dtrain.txt


In [1]:
1

1

In [3]:
help(xgb.XGBClassifier)
# clf.fit(xgb_model=) # loaded before training (allows training continuation).
clf = xgb.XGBClassifier().load_model(fname='') # feature names not loaded 
# can also clf.save_model() # but they say in the doc... 
# **If you are using only the Python interface, we recommend pickling the
#      model object for best results.**
#


Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  Parameters
 |  ----------
 |  max_depth : int
 |      Maximum tree depth for base learners.
 |  learning_rate : float
 |      Boosting learning rate (xgb's "eta")
 |  n_estimators : int
 |      Number of trees to fit.
 |  verbosity : int
 |      The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |  silent : boolean
 |      Whether to print messages while running boosting. Deprecat

In [12]:
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
X_train, X_test, y_train, y_test = train_test_split(X, y)

clf = xgb.XGBClassifier()


# preproc
(X_transformed,
     one_hot_enc, le,
     y_enc) = pv1.preprocess(X_train, y_train, # X[train_index]
                         neighborhoods)
labels = le.classes_

# Test set
X_test_transformed = one_hot_enc.transform(X_test)
y_test_enc = le.transform(y_test)

In [20]:
len(Counter(y_train)) # 54

54

In [13]:
# pre-evaluate
# y_prob_vec = clf.predict_proba(X_test_transformed)
# loss = fu.big_logloss(y_test_enc, y_prob_vec, labels)
#
# ok, cannot call on raw model like TensorFlow. heh.
# XGBoostError: need to call fit or load_model beforehand

XGBoostError: need to call fit or load_model beforehand

In [17]:
parts = fu.get_partitions(list(range(X_transformed.shape[0])), slice_size=1000)
loss_vec = []
for part in tqdm(parts):
    i, j = part[0], part[-1]   

    clf.fit(X_transformed[i:j], y_enc[i:j])
    
    y_prob_vec = clf.predict_proba(X_test_transformed)
    loss = fu.big_logloss(y_test_enc, y_prob_vec, labels)
    loss_vec.append(loss)

ValueError: The number of classes in labels is different from that in y_pred. Classes found in labels: ['Alphabet City' 'Battery Park City' 'Bedford-Stuyvesant' 'Boerum Hill'
 'Bowery' 'Brooklyn Heights' 'Brooklyn Navy Yard' 'Central Park' 'Chelsea'
 'Chinatown' 'Civic Center' 'Clinton Hill'
 'Columbia Street Waterfront District' 'Downtown Brooklyn' 'Dumbo'
 'East Village' 'Financial District' 'Flatiron District' 'Fort Greene'
 'Fulton Ferry District' 'Garment District' 'Gramercy Park'
 'Greenwich Village' "Hell's Kitchen" 'Hudson Square' 'Kips Bay'
 'Korea Town' 'Lincoln Square' 'Little Italy' 'Lower East Side'
 'Lower Manhattan' 'Meatpacking District' 'Midtown' 'Midtown East'
 'Midtown West' 'Murray Hill' 'NoHo' 'NoMad' 'Nolita' 'Park Slope'
 'Peter Cooper Village' 'Rose Hill' 'SoHo' 'Stuyvesant Town'
 'Sutton Place' 'Theater District' 'Tribeca' 'Tudor City' 'Two Bridges'
 'Ukrainian Village' 'Union Square' 'Vinegar Hill' 'West Village'
 'Williamsburg']

In [22]:
#loss = fu.big_logloss(y_test_enc, y_prob_vec, labels)
len(labels), y_test_enc.shape, y_prob_vec.shape, len(Counter(y_enc[i:j]))

(54, (210854,), (210854, 53), 53)

In [28]:
# Ok... going to try shuffling first instead.. 
# so the len(Counter(y_enc[i:j])) ,  hopefully , of the first fit , has 54 not 53.

# Ok I shuffled about four times until I ended up where the first part below here was '54'

size = X_transformed.shape[0]
print(size)
indices = np.random.choice(range(size), size=size, replace=False)
# X_train_shfl = X_transformed[indices]
# Ylabels_train_shfl = y_enc[indices]#.astype('int64')

parts = fu.get_partitions(indices, slice_size=1000)
len(parts[0])
len(Counter(y_enc[parts[0]]))

632562


54

In [29]:
workdir = fu.make_work_dir()

In [32]:
print("workdir", workdir); fu.log(workdir, 'First line')

workdir /opt/program/artifacts/2020-06-16T033314Z


In [33]:
fu.log(workdir, 'Starting')
loss_vec = []
for i, part in enumerate(tqdm(parts)):
    clf.fit(X_transformed[part], y_enc[part])
    fu.log(workdir, f'[{i}] Done fit')
    
    y_prob_vec = clf.predict_proba(X_test_transformed)
    fu.log(workdir, f'[{i}] Done predict_proba')
    
    loss = fu.big_logloss(y_test_enc, y_prob_vec, labels)
    fu.log(workdir, f'[{i}] Done big_logloss')
    
    loss_vec.append(loss)

ValueError: The number of classes in labels is different from that in y_pred. Classes found in labels: ['Alphabet City' 'Battery Park City' 'Bedford-Stuyvesant' 'Boerum Hill'
 'Bowery' 'Brooklyn Heights' 'Brooklyn Navy Yard' 'Central Park' 'Chelsea'
 'Chinatown' 'Civic Center' 'Clinton Hill'
 'Columbia Street Waterfront District' 'Downtown Brooklyn' 'Dumbo'
 'East Village' 'Financial District' 'Flatiron District' 'Fort Greene'
 'Fulton Ferry District' 'Garment District' 'Gramercy Park'
 'Greenwich Village' "Hell's Kitchen" 'Hudson Square' 'Kips Bay'
 'Korea Town' 'Lincoln Square' 'Little Italy' 'Lower East Side'
 'Lower Manhattan' 'Meatpacking District' 'Midtown' 'Midtown East'
 'Midtown West' 'Murray Hill' 'NoHo' 'NoMad' 'Nolita' 'Park Slope'
 'Peter Cooper Village' 'Rose Hill' 'SoHo' 'Stuyvesant Town'
 'Sutton Place' 'Theater District' 'Tribeca' 'Tudor City' 'Two Bridges'
 'Ukrainian Village' 'Union Square' 'Vinegar Hill' 'West Village'
 'Williamsburg']

In [ ]:
# Observing here, this first log line from the file here , "2020-06-16 03:36:20Z, Starting" , 
# and I see that the big_logloss() first part  did not crash here. 
# ...   but loss vec ... hmm alll zeroes... not learning? 

In [34]:
loss_vec

[0.0, 0.0]

In [35]:
y_test_enc.shape, y_prob_vec.shape, labels.shape

((210854,), (210854, 52), (54,))

In [ ]:
# Since the output of y_prob_vec is showing a size of 52, this makes me think the fit() is starint from scratch
# ... I can at least quickly try out that save ...

In [ ]:
workdir = fu.make_work_dir(); print(workdir)
fu.log(workdir, 'Starting')
prev_model = None
loss_vec = []; acc_vec = []
for i, part in enumerate(tqdm(parts)):
    clf.fit(X_transformed[part], y_enc[part], xgb_model=prev_model)
    fu.log(workdir, f'[{i}] Done fit')
    
    prev_model = f'{workdir}/model.xg'
    clf.save_model(prev_model)
    
    y_prob_vec = clf.predict_proba(X_test_transformed)
    fu.log(workdir, f'[{i}] Done predict_proba')
    
    loss = fu.big_logloss(y_test_enc, y_prob_vec, labels)
    fu.log(workdir, f'[{i}] Done big_logloss, loss={loss}.')
    
    loss_vec.append(loss)
    
    acc = accuracy_score(y_test_enc, np.argmax(y_prob_vec, axis=1))
    acc_vec.append(acc)
    fu.log(workdir, f'[{i}] Done accuracy, acc={acc}.')

/opt/program/artifacts/2020-06-16T035733Z



```
(pandars3) $ tail -f artifacts/2020-06-16T035733Z/work.log 
2020-06-16 03:57:33Z, Starting
2020-06-16 03:57:35Z, [0] Done fit
2020-06-16 03:59:10Z, [0] Done predict_proba
2020-06-16 03:59:10Z, [0] Done big_logloss, loss=0.0.
2020-06-16 03:59:10Z, [0] Done accuracy, acc=0.11610403407096853.
2020-06-16 03:59:13Z, [1] Done fit
2020-06-16 04:02:27Z, [1] Done predict_proba
2020-06-16 04:02:27Z, [1] Done big_logloss, loss=0.0.
2020-06-16 04:02:27Z, [1] Done accuracy, acc=0.11933849962533317.
2020-06-16 04:02:31Z, [2] Done fit
2020-06-16 04:09:21Z, [2] Done predict_proba
2020-06-16 04:09:21Z, [2] Done big_logloss, loss=0.0.
2020-06-16 04:09:21Z, [2] Done accuracy, acc=0.11234313790584954.
2020-06-16 04:09:27Z, [3] Done fit
2020-06-16 04:19:43Z, [3] Done predict_proba
2020-06-16 04:19:43Z, [3] Done big_logloss, loss=0.0.
2020-06-16 04:19:43Z, [3] Done accuracy, acc=0.0981295114154818.
2020-06-16 04:19:52Z, [4] Done fit
2020-06-16 04:35:20Z, [4] Done predict_proba
2020-06-16 04:35:20Z, [4] Done big_logloss, loss=0.0.
2020-06-16 04:35:20Z, [4] Done accuracy, acc=0.015944682102307758.
2020-06-16 04:35:31Z, [5] Done fit
2020-06-16 04:56:32Z, [5] Done predict_proba
2020-06-16 04:56:33Z, [5] Done big_logloss, loss=0.0.
2020-06-16 04:56:33Z, [5] Done accuracy, acc=0.10582678061597124.
2020-06-16 04:56:47Z, [6] Done fit
2020-06-16 05:22:41Z, [6] Done predict_proba
2020-06-16 05:22:41Z, [6] Done big_logloss, loss=0.0.
2020-06-16 05:22:41Z, [6] Done accuracy, acc=0.109895946958559.
2020-06-16 05:22:58Z, [7] Done fit
2020-06-16 05:53:38Z, [7] Done predict_proba
2020-06-16 05:53:38Z, [7] Done big_logloss, loss=0.0.
2020-06-16 05:53:38Z, [7] Done accuracy, acc=0.1033795896686807.
2020-06-16 05:53:58Z, [8] Done fit
2020-06-16 06:29:45Z, [8] Done predict_proba
2020-06-16 06:29:45Z, [8] Done big_logloss, loss=0.0.
2020-06-16 06:29:45Z, [8] Done accuracy, acc=0.020260464586870537.
2020-06-16 06:30:08Z, [9] Done fit
2020-06-16 07:10:49Z, [9] Done predict_proba
2020-06-16 07:10:49Z, [9] Done big_logloss, loss=0.0.
2020-06-16 07:10:49Z, [9] Done accuracy, acc=0.12078499815037894.
2020-06-16 07:11:14Z, [10] Done fit
2020-06-16 07:57:28Z, [10] Done predict_proba
2020-06-16 07:57:28Z, [10] Done big_logloss, loss=0.0.
2020-06-16 07:57:28Z, [10] Done accuracy, acc=0.12298557295569446.
2020-06-16 07:57:56Z, [11] Done fit
2020-06-16 08:49:22Z, [11] Done predict_proba
2020-06-16 08:49:23Z, [11] Done big_logloss, loss=0.0.
2020-06-16 08:49:23Z, [11] Done accuracy, acc=0.1153736708812733.
2020-06-16 08:49:54Z, [12] Done fit
2020-06-16 09:47:09Z, [12] Done predict_proba
2020-06-16 09:47:09Z, [12] Done big_logloss, loss=0.0.
2020-06-16 09:47:09Z, [12] Done accuracy, acc=0.11813861724226242.
2020-06-16 09:47:44Z, [13] Done fit
2020-06-16 10:50:05Z, [13] Done predict_proba
2020-06-16 10:50:06Z, [13] Done big_logloss, loss=0.0.
2020-06-16 10:50:06Z, [13] Done accuracy, acc=0.11920570631811585.
2020-06-16 10:50:45Z, [14] Done fit
2020-06-16 12:02:35Z, [14] Done predict_proba
2020-06-16 12:02:35Z, [14] Done big_logloss, loss=0.0.
2020-06-16 12:02:35Z, [14] Done accuracy, acc=0.11395088544680206.
2020-06-16 12:03:16Z, [15] Done fit
2020-06-16 13:22:31Z, [15] Done predict_proba
2020-06-16 13:22:31Z, [15] Done big_logloss, loss=0.0.
2020-06-16 13:22:31Z, [15] Done accuracy, acc=0.11543532491676706.
2020-06-16 13:23:15Z, [16] Done fit
2020-06-16 14:47:58Z, [16] Done predict_proba
2020-06-16 14:47:58Z, [16] Done big_logloss, loss=0.0.
2020-06-16 14:47:58Z, [16] Done accuracy, acc=0.026767336640519033.
2020-06-16 14:48:46Z, [17] Done fit
2020-06-16 16:19:31Z, [17] Done predict_proba
2020-06-16 16:19:32Z, [17] Done big_logloss, loss=0.0.
2020-06-16 16:19:32Z, [17] Done accuracy, acc=0.11969419598395098.
2020-06-16 16:20:23Z, [18] Done fit



```


In [1]:

# still not sure why loss 0 tho. 